# Getting started with Ibis

In the [previous notebook](./00%20-%20Welcome.ipynb), we created a DuckDB database file with the [nycflights13 data](https://github.com/hadley/nycflights13). DuckDB is fast and runs locally, so it's handy for lots of use cases, including tutorials. Let's begin by importing Ibis and connecting to the database.

In [1]:
import ibis

con = ibis.duckdb.connect("nycflights13.ddb", read_only=True)

**Note**: When you connect to a DuckDB database file, DuckDB creates a WAL file to prevent data corruption. If you see a `nycflights13.ddb.wal` file, you can safely ignore it. It will get cleaned up automatically.

Now we have a connection, we can start by looking around. Are there any tables in this database?

In [2]:
con.list_tables()

['flights', 'weather']

Two, in fact! Let's take a look at the `flights` table first.

In [3]:
flights = con.table("flights")

By default, you'll get a printable representation of the table schema, showing the name and data type of each column.

In [4]:
flights

DatabaseTable: flights
  year           int64
  month          int64
  day            int64
  dep_time       string
  sched_dep_time int64
  dep_delay      string
  arr_time       string
  sched_arr_time int64
  arr_delay      string
  carrier        string
  flight         int64
  tailnum        string
  origin         string
  dest           string
  air_time       string
  distance       int64
  hour           int64
  minute         int64
  time_hour      timestamp(6)

If we call the `head` method to peek at the data, you'll notice that we don't actually see data (yet); what's going on?

In [5]:
flights.head()

r0 := DatabaseTable: flights
  year           int64
  month          int64
  day            int64
  dep_time       string
  sched_dep_time int64
  dep_delay      string
  arr_time       string
  sched_arr_time int64
  arr_delay      string
  carrier        string
  flight         int64
  tailnum        string
  origin         string
  dest           string
  air_time       string
  distance       int64
  hour           int64
  minute         int64
  time_hour      timestamp(6)

Limit[r0, n=5]

Ibis has a deferred execution model. It builds up expressions based on what you ask it to do, and then executes those expressions on request.

In this case, our query isn't too involved; we want to see the first few rows of the `flights` table. We can do that by asking for the results of this query as a `pandas.DataFrame`:

In [6]:
flights.head().to_pandas()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517,515,2,830,819,11,UA,1545,N14228,EWR,IAH,227,1400,5,15,2013-01-01 10:00:00
1,2013,1,1,533,529,4,850,830,20,UA,1714,N24211,LGA,IAH,227,1416,5,29,2013-01-01 10:00:00
2,2013,1,1,542,540,2,923,850,33,AA,1141,N619AA,JFK,MIA,160,1089,5,40,2013-01-01 10:00:00
3,2013,1,1,544,545,-1,1004,1022,-18,B6,725,N804JB,JFK,BQN,183,1576,5,45,2013-01-01 10:00:00
4,2013,1,1,554,600,-6,812,837,-25,DL,461,N668DN,LGA,ATL,116,762,6,0,2013-01-01 11:00:00


Or a `pyarrow.Table`:

In [7]:
flights.head().to_pyarrow()

pyarrow.Table
year: int64
month: int64
day: int64
dep_time: string
sched_dep_time: int64
dep_delay: string
arr_time: string
sched_arr_time: int64
arr_delay: string
carrier: string
flight: int64
tailnum: string
origin: string
dest: string
air_time: string
distance: int64
hour: int64
minute: int64
time_hour: timestamp[us]
----
year: [[2013,2013,2013,2013,2013]]
month: [[1,1,1,1,1]]
day: [[1,1,1,1,1]]
dep_time: [["517","533","542","544","554"]]
sched_dep_time: [[515,529,540,545,600]]
dep_delay: [["2","4","2","-1","-6"]]
arr_time: [["830","850","923","1004","812"]]
sched_arr_time: [[819,830,850,1022,837]]
arr_delay: [["11","20","33","-18","-25"]]
carrier: [["UA","UA","AA","B6","DL"]]
...

Or a `polars.DataFrame`:

In [8]:
flights.head().to_polars()

year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
i64,i64,i64,str,i64,str,str,i64,str,str,i64,str,str,str,str,i64,i64,i64,datetime[ns]
2013,1,1,"""517""",515,"""2""","""830""",819,"""11""","""UA""",1545,"""N14228""","""EWR""","""IAH""","""227""",1400,5,15,2013-01-01 10:00:00
2013,1,1,"""533""",529,"""4""","""850""",830,"""20""","""UA""",1714,"""N24211""","""LGA""","""IAH""","""227""",1416,5,29,2013-01-01 10:00:00
2013,1,1,"""542""",540,"""2""","""923""",850,"""33""","""AA""",1141,"""N619AA""","""JFK""","""MIA""","""160""",1089,5,40,2013-01-01 10:00:00
2013,1,1,"""544""",545,"""-1""","""1004""",1022,"""-18""","""B6""",725,"""N804JB""","""JFK""","""BQN""","""183""",1576,5,45,2013-01-01 10:00:00
2013,1,1,"""554""",600,"""-6""","""812""",837,"""-25""","""DL""",461,"""N668DN""","""LGA""","""ATL""","""116""",762,6,0,2013-01-01 11:00:00


We'll get into more detail about what Ibis is doing a bit later on. For now, the important point is that Ibis is deferred.

## Interactive mode

Remember when we said Ibis is deferred? Sometimes you want eager execution so you can explore a dataset. For the rest of this notebook, we'll turn on interactive mode, where Ibis will eagerly execute as much of the query as it needs to in order to show you the first 10 rows of the result.

In [9]:
ibis.options.interactive = True

In interactive mode, we use `rich` to render the output inline:

In [10]:
flights.head()

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

## Tables and columns

`flights` is a table!  A table is a collection of one or more columns, each with a specific datatype.

In [11]:
flights

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

In [12]:
type(flights)

ibis.expr.types.relations.Table

We can look at a single column of that table using the column name as an attribute:

In [13]:
flights.carrier

┏━━━━━━━━━┓
┃ carrier ┃
┡━━━━━━━━━┩
│ string  │
├─────────┤
│ UA      │
│ UA      │
│ AA      │
│ B6      │
│ DL      │
│ UA      │
│ B6      │
│ EV      │
│ B6      │
│ AA      │
│ …       │
└─────────┘

What kind of column is `carrier`? It's a `StringColumn`!

In [14]:
type(flights.carrier)

ibis.expr.types.strings.StringColumn

## Ibis "verbs", or, stuff you can do to a table

The rest of this notebook covers some of the general methods you can use to alter the output of a particular table.

We'll cover, in order, `filter`, `select`, `drop`, `mutate`, `order_by`, `aggregate`, and `group_by`. Time to dive in!

## Filter

A filter allows you to view a subset of the rows in a table, based on some condition.

For instance, we might want to only view data for JetBlue flights:

In [15]:
flights.filter(flights.carrier == "B6")

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 849      │            851 │ -2        │ B6      │     49 │ N793JB  │ JFK    │ PBI    │ 149      │     1028 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 853      │            856 │ -3        │ B6      │     71 │ N657JB  │ JFK    │ TPA    │ 158      │     1005 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 559      │            559 │ 0         │ 702      │            706 │ -4        │ B6      │   1806 │ N708JB  │ JFK    │ BOS    │ 44       │      187 │     5 │     59 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 600      │            600 │ 0         │ 851      │            858 │ -7        │ B6      │    371 │ N595JB  │ LGA    │ FLL    │ 152      │     1076 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 601      │            600 │ 1         │ 844      │            850 │ -6        │ B6      │    343 │ N644JB  │ EWR    │ PBI    │ 147      │     1023 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 613      │            610 │ 3         │ 925      │            921 │ 4         │ B6      │    135 │ N635JB  │ JFK    │ RSW    │ 175      │     1074 │     6 │     10 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 615      │            615 │ 0         │ 1039     │           1100 │ -21       │ B6      │    709 │ N794JB  │ JFK    │ SJU    │ 182      │     1598 │     6 │     15 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

You can also combine multiple filters, across multiple columns.

We can subset the data down to JetBlue flights from JFK:

In [16]:
expr = flights.filter((flights.carrier == "B6") & (flights.origin == "JFK"))
expr

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 849      │            851 │ -2        │ B6      │     49 │ N793JB  │ JFK    │ PBI    │ 149      │     1028 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 853      │            856 │ -3        │ B6      │     71 │ N657JB  │ JFK    │ TPA    │ 158      │     1005 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 559      │            559 │ 0         │ 702      │            706 │ -4        │ B6      │   1806 │ N708JB  │ JFK    │ BOS    │ 44       │      187 │     5 │     59 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 613      │            610 │ 3         │ 925      │            921 │ 4         │ B6      │    135 │ N635JB  │ JFK    │ RSW    │ 175      │     1074 │     6 │     10 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 615      │            615 │ 0         │ 1039     │           1100 │ -21       │ B6      │    709 │ N794JB  │ JFK    │ SJU    │ 182      │     1598 │     6 │     15 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 639      │            640 │ -1        │ 739      │            749 │ -10       │ B6      │   1002 │ N805JB  │ JFK    │ BOS    │ 41       │      187 │     6 │     40 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 645      │            647 │ -2        │ 815      │            810 │ 5         │ B6      │    102 │ N796JB  │ JFK    │ BUF    │ 63       │      301 │     6 │     47 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 651      │            655 │ -4        │ 936      │            942 │ -6        │ B6      │    203 │ N558JB  │ JFK    │ LAS    │ 323      │     2248 │     6 │     55 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

Above, we combined two filters using `&`. You can also pass them in as individual arguments:

In [17]:
expr = flights.filter(
    flights.carrier == "B6",
    flights.origin == "JFK",
)
expr

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 849      │            851 │ -2        │ B6      │     49 │ N793JB  │ JFK    │ PBI    │ 149      │     1028 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 853      │            856 │ -3        │ B6      │     71 │ N657JB  │ JFK    │ TPA    │ 158      │     1005 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 559      │            559 │ 0         │ 702      │            706 │ -4        │ B6      │   1806 │ N708JB  │ JFK    │ BOS    │ 44       │      187 │     5 │     59 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 613      │            610 │ 3         │ 925      │            921 │ 4         │ B6      │    135 │ N635JB  │ JFK    │ RSW    │ 175      │     1074 │     6 │     10 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 615      │            615 │ 0         │ 1039     │           1100 │ -21       │ B6      │    709 │ N794JB  │ JFK    │ SJU    │ 182      │     1598 │     6 │     15 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 639      │            640 │ -1        │ 739      │            749 │ -10       │ B6      │   1002 │ N805JB  │ JFK    │ BOS    │ 41       │      187 │     6 │     40 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 645      │            647 │ -2        │ 815      │            810 │ 5         │ B6      │    102 │ N796JB  │ JFK    │ BUF    │ 63       │      301 │     6 │     47 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 651      │            655 │ -4        │ 936      │            942 │ -6        │ B6      │    203 │ N558JB  │ JFK    │ LAS    │ 323      │     2248 │     6 │     55 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

In [18]:
ibis.to_sql(expr)

```sql
SELECT
  *
FROM "flights" AS "t0"
WHERE
  "t0"."carrier" = 'B6' AND "t0"."origin" = 'JFK'
```

## Select

Filter filters, Select selects (there's a pattern here).
If you only want a subset of the columns in the original table, you can select
those columns explicitly.

You can refer to the columns using strings:

In [19]:
flights.select("carrier", "origin", "dest")

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ carrier ┃ origin ┃ dest   ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string  │ string │ string │
├─────────┼────────┼────────┤
│ UA      │ EWR    │ IAH    │
│ UA      │ LGA    │ IAH    │
│ AA      │ JFK    │ MIA    │
│ B6      │ JFK    │ BQN    │
│ DL      │ LGA    │ ATL    │
│ UA      │ EWR    │ ORD    │
│ B6      │ EWR    │ FLL    │
│ EV      │ LGA    │ IAD    │
│ B6      │ JFK    │ MCO    │
│ AA      │ LGA    │ ORD    │
│ …       │ …      │ …      │
└─────────┴────────┴────────┘

Or you can use explicit references to the `Column` objects:

In [20]:
flights.select(flights.carrier, flights.origin, flights.dest)

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ carrier ┃ origin ┃ dest   ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string  │ string │ string │
├─────────┼────────┼────────┤
│ UA      │ EWR    │ IAH    │
│ UA      │ LGA    │ IAH    │
│ AA      │ JFK    │ MIA    │
│ B6      │ JFK    │ BQN    │
│ DL      │ LGA    │ ATL    │
│ UA      │ EWR    │ ORD    │
│ B6      │ EWR    │ FLL    │
│ EV      │ LGA    │ IAD    │
│ B6      │ JFK    │ MCO    │
│ AA      │ LGA    │ ORD    │
│ …       │ …      │ …      │
└─────────┴────────┴────────┘

Or you can mix and match:

In [21]:
flights.select("carrier", "origin", flights.dest)

┏━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃ carrier ┃ origin ┃ dest   ┃
┡━━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ string  │ string │ string │
├─────────┼────────┼────────┤
│ UA      │ EWR    │ IAH    │
│ UA      │ LGA    │ IAH    │
│ AA      │ JFK    │ MIA    │
│ B6      │ JFK    │ BQN    │
│ DL      │ LGA    │ ATL    │
│ UA      │ EWR    │ ORD    │
│ B6      │ EWR    │ FLL    │
│ EV      │ LGA    │ IAD    │
│ B6      │ JFK    │ MCO    │
│ AA      │ LGA    │ ORD    │
│ …       │ …      │ …      │
└─────────┴────────┴────────┘

## Drop

Drop is nearly the same as Select, but rather than explicitly choosing the columns to display, we explicitly choose the columns to _not_ display.

And as with `select`, you can specify the columns as strings:

In [22]:
flights.drop("flight", "tailnum")

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

Or you can use explicit references to the `Column` objects:

In [23]:
flights.drop(flights.flight, flights.tailnum)

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

Or you can mix and match:

In [24]:
flights.drop("flight", flights.tailnum)

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

## Mutate

Everything we've seen so far has been subtractive—removing rows or columns. What about _adding_ columns?

That's what `mutate` is for! You can create a new column as a function of other existing columns (for example, converting units):

In [25]:
flights.mutate(distance_km=flights.distance * 1.609)

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃ distance_km ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │ float64     │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┼─────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │    2252.600 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │    2278.344 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │    1752.201 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │    2535.784 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │    1226.058 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │    1156.871 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │    1713.585 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │     368.461 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │    1518.896 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │    1179.397 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │           … │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┴─────────────┘

Or you can create a new column and populate it with some literal value:

In [26]:
flights.mutate(my_favorite_number=ibis.literal(41))

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃ my_favorite_number ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │ int8               │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┼────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │                 41 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │                 41 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │                 41 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │                 41 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │                 41 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │                 41 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │                 41 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │                 41 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │                 41 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │                 41 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │                  … │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┴────────────────────┘

## On immutability

We've filtered, selected, dropped, and mutated this `flights` table quite a bit.

In [27]:
flights

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ distance ┃ hour  ┃ minute ┃ time_hour           ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64    │ int64 │ int64  │ timestamp(6)        │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼──────────┼───────┼────────┼─────────────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     1400 │     5 │     15 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     1416 │     5 │     29 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     1089 │     5 │     40 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     1576 │     5 │     45 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │      762 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │      719 │     5 │     58 │ 2013-01-01 10:00:00 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     1065 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │      229 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │      944 │     6 │      0 │ 2013-01-01 11:00:00 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │      733 │     6 │      0 │ 2013-01-01 11:00:00 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │        … │     … │      … │ …                   │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴──────────┴───────┴────────┴─────────────────────┘

And yet, notice that none of our changes persist—the base table for our query isn't altered. The query (or expression) is a recipe of things to do with the base table (`flights`).

If you want to keep an expression around, you can assign it to a variable:

In [28]:
expr = flights.select("carrier", "origin")
expr

┏━━━━━━━━━┳━━━━━━━━┓
┃ carrier ┃ origin ┃
┡━━━━━━━━━╇━━━━━━━━┩
│ string  │ string │
├─────────┼────────┤
│ UA      │ EWR    │
│ UA      │ LGA    │
│ AA      │ JFK    │
│ B6      │ JFK    │
│ DL      │ LGA    │
│ UA      │ EWR    │
│ B6      │ EWR    │
│ EV      │ LGA    │
│ B6      │ JFK    │
│ AA      │ LGA    │
│ …       │ …      │
└─────────┴────────┘

**Note**: Every time you execute an expression (via interactive mode, or `to_pandas`, or similar), the entire expression gets executed, starting from the base table.  DuckDB is very fast and this dataset is very small, so the delay is unnoticeable, but for very large datasets, it might become more pronounced.  There is functionality to `cache` intermediate results that isn't covered in this tutorial, but you can [read more about it in the docs](https://ibis-project.org/reference/expression-tables.html#ibis.expr.types.relations.Table.cache).

## Method chaining

You can build up complicated queries by chaining together Ibis methods. The output of many Ibis methods is a table (just like `flights`!) and we can continue calling table methods until we're satisfied. Or until we end up with something that _isn't_ a table. More on that later.

In [29]:
flights.select("carrier", "origin", "dest").drop("carrier")

┏━━━━━━━━┳━━━━━━━━┓
┃ origin ┃ dest   ┃
┡━━━━━━━━╇━━━━━━━━┩
│ string │ string │
├────────┼────────┤
│ EWR    │ IAH    │
│ LGA    │ IAH    │
│ JFK    │ MIA    │
│ JFK    │ BQN    │
│ LGA    │ ATL    │
│ EWR    │ ORD    │
│ EWR    │ FLL    │
│ LGA    │ IAD    │
│ JFK    │ MCO    │
│ LGA    │ ORD    │
│ …      │ …      │
└────────┴────────┘

Not the most complicated (or useful) query, but we'll see more soon.

### Exercise 1

Convert the `distance` column from miles to kilometers. For an approximate result, multiply by 1.609.

Two ways you might accomplish this:

- Chaining `.mutate` to create the new column and `.drop` to drop the original imperial column
- Using a single `.select` to create the new column as well as select the remaining columns

Try both ways below! How do they compare?

In [30]:
# Convert the imperial units to metric, and drop the imperial columns.
# Try this using a `.mutate` and `.drop` call.
flights_metric_mutate_drop = flights

In [31]:
# Convert the imperial units to metric, and drop the imperial columns.
# Try this using a single `.select` call.
flights_metric_select = flights

#### Solutions

In [33]:
# %load solutions/nb01_ex01_mutate_drop.py
# Convert the imperial units to metric, and drop the imperial columns.
flights_metric_mutate_drop = (
    flights.mutate(distance_km=flights.distance * 1.609).drop("distance")
)

flights_metric_mutate_drop


┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ hour  ┃ minute ┃ time_hour           ┃ distance_km ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64 │ int64  │ timestamp(6)        │ float64     │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼───────┼────────┼─────────────────────┼─────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     5 │     15 │ 2013-01-01 10:00:00 │    2252.600 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     5 │     29 │ 2013-01-01 10:00:00 │    2278.344 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     5 │     40 │ 2013-01-01 10:00:00 │    1752.201 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     5 │     45 │ 2013-01-01 10:00:00 │    2535.784 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │     6 │      0 │ 2013-01-01 11:00:00 │    1226.058 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │     5 │     58 │ 2013-01-01 10:00:00 │    1156.871 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     6 │      0 │ 2013-01-01 11:00:00 │    1713.585 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │     6 │      0 │ 2013-01-01 11:00:00 │     368.461 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │     6 │      0 │ 2013-01-01 11:00:00 │    1518.896 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │     6 │      0 │ 2013-01-01 11:00:00 │    1179.397 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │     … │      … │ …                   │           … │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴───────┴────────┴─────────────────────┴─────────────┘

In [35]:
# %load solutions/nb01_ex01_select.py
# Convert the imperial units to metric, and drop the imperial columns.
flights_metric_select = flights.select(
    "year",
    "month",
    "day",
    "dep_time",
    "sched_dep_time",
    "dep_delay",
    "arr_time",
    "sched_arr_time",
    "arr_delay",
    "carrier",
    "flight",
    "tailnum",
    "origin",
    "dest",
    "air_time",
    "hour",
    "minute",
    "time_hour",
    distance_km=flights.distance * 1.609,
)

flights_metric_select


┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ year  ┃ month ┃ day   ┃ dep_time ┃ sched_dep_time ┃ dep_delay ┃ arr_time ┃ sched_arr_time ┃ arr_delay ┃ carrier ┃ flight ┃ tailnum ┃ origin ┃ dest   ┃ air_time ┃ hour  ┃ minute ┃ time_hour           ┃ distance_km ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ int64 │ int64 │ string   │ int64          │ string    │ string   │ int64          │ string    │ string  │ int64  │ string  │ string │ string │ string   │ int64 │ int64  │ timestamp(6)        │ float64     │
├───────┼───────┼───────┼──────────┼────────────────┼───────────┼──────────┼────────────────┼───────────┼─────────┼────────┼─────────┼────────┼────────┼──────────┼───────┼────────┼─────────────────────┼─────────────┤
│  2013 │     1 │     1 │ 517      │            515 │ 2         │ 830      │            819 │ 11        │ UA      │   1545 │ N14228  │ EWR    │ IAH    │ 227      │     5 │     15 │ 2013-01-01 10:00:00 │    2252.600 │
│  2013 │     1 │     1 │ 533      │            529 │ 4         │ 850      │            830 │ 20        │ UA      │   1714 │ N24211  │ LGA    │ IAH    │ 227      │     5 │     29 │ 2013-01-01 10:00:00 │    2278.344 │
│  2013 │     1 │     1 │ 542      │            540 │ 2         │ 923      │            850 │ 33        │ AA      │   1141 │ N619AA  │ JFK    │ MIA    │ 160      │     5 │     40 │ 2013-01-01 10:00:00 │    1752.201 │
│  2013 │     1 │     1 │ 544      │            545 │ -1        │ 1004     │           1022 │ -18       │ B6      │    725 │ N804JB  │ JFK    │ BQN    │ 183      │     5 │     45 │ 2013-01-01 10:00:00 │    2535.784 │
│  2013 │     1 │     1 │ 554      │            600 │ -6        │ 812      │            837 │ -25       │ DL      │    461 │ N668DN  │ LGA    │ ATL    │ 116      │     6 │      0 │ 2013-01-01 11:00:00 │    1226.058 │
│  2013 │     1 │     1 │ 554      │            558 │ -4        │ 740      │            728 │ 12        │ UA      │   1696 │ N39463  │ EWR    │ ORD    │ 150      │     5 │     58 │ 2013-01-01 10:00:00 │    1156.871 │
│  2013 │     1 │     1 │ 555      │            600 │ -5        │ 913      │            854 │ 19        │ B6      │    507 │ N516JB  │ EWR    │ FLL    │ 158      │     6 │      0 │ 2013-01-01 11:00:00 │    1713.585 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 709      │            723 │ -14       │ EV      │   5708 │ N829AS  │ LGA    │ IAD    │ 53       │     6 │      0 │ 2013-01-01 11:00:00 │     368.461 │
│  2013 │     1 │     1 │ 557      │            600 │ -3        │ 838      │            846 │ -8        │ B6      │     79 │ N593JB  │ JFK    │ MCO    │ 140      │     6 │      0 │ 2013-01-01 11:00:00 │    1518.896 │
│  2013 │     1 │     1 │ 558      │            600 │ -2        │ 753      │            745 │ 8         │ AA      │    301 │ N3ALAA  │ LGA    │ ORD    │ 138      │     6 │      0 │ 2013-01-01 11:00:00 │    1179.397 │
│     … │     … │     … │ …        │              … │ …         │ …        │              … │ …         │ …       │      … │ …       │ …      │ …      │ …        │     … │      … │ …                   │           … │
└───────┴───────┴───────┴──────────┴────────────────┴───────────┴──────────┴────────────────┴───────────┴─────────┴────────┴─────────┴────────┴────────┴──────────┴───────┴────────┴─────────────────────┴─────────────┘

#### Does it matter which method you choose?

In this case, no. Sometimes, there might be a small difference in the generated SQL, but they will be semantically equivalent.

In [ ]:
ibis.to_sql(flights_metric_mutate_drop)

In [ ]:
ibis.to_sql(flights_metric_select)

In practice, small differences in the generated SQL don't make a difference. Any modern SQL execution engine will optimize variations to the same set of operations, and there will be no measurable performance difference.

## Order by

Want to order your data by a given column or columns?  Use `order_by`!

The default ordering direction is ascending:

In [ ]:
flights.order_by(flights.distance)

We can ask Ibis to sort in descending order, too.

In [ ]:
flights.order_by(flights.distance.desc())

Let's select out a subset of the columns to keep this a bit tidier.

In [ ]:
flights.order_by(flights.distance.desc()).select(
    "carrier", "origin", "dest", "distance"
)

You can also call `ibis.desc` on the column name to set the order direction:

In [ ]:
flights.order_by(ibis.desc("distance")).select("carrier", "origin", "dest", "distance")

## Aggregate

Ibis has several aggregate functions available to help summarize data.  All the old favorites are there: `mean`, `max`, `min`, `count`, `sum`...

You can aggregate a column by calling the method on that column:

In [ ]:
flights.distance.mean()

Or you can compute multiple aggregates using the `aggregate` method (also
available as `agg` for faster typing):

In [ ]:
flights.agg([flights.distance.mean(), flights.air_time.min()])

If you don't like the column names Ibis generates for you, choose your own!

In [ ]:
flights.agg(
    average_distance=flights.distance.mean(),
    shortest_air_time=flights.air_time.min(),
)

But aggregates really shine when paired with a `group_by`!

## Group by

`group_by` creates groupings of rows that have the same value for one or more columns.

But it doesn't do much on its own—you can pair it with `agg` to get a result.

In [ ]:
flights.group_by("carrier").agg()

Without any aggregate function specified, we get the distinct values of the grouped column.

We can add a second column to the `group_by` to get the distinct pairs across both columns:

In [ ]:
flights.group_by(["carrier", "origin"]).agg()

Now, if we add an aggregation function to that, we start to really open things up.

In [ ]:
flights.group_by(["carrier", "origin"]).agg(flights.distance.mean())

By adding that `mean` to the `aggregate`, we now have a concise way to calculate aggregates over each of the distinct groups in the `group_by`. And we can calculate as many aggregates as we need.

In [ ]:
flights.group_by(["carrier", "origin"]).agg(
    [flights.distance.mean(), flights.air_time.min()]
)

If we need more specific groups, we can add to the `group_by`.

In [ ]:
flights.group_by(["carrier", "origin", "dest"]).agg(
    [flights.distance.mean(), flights.air_time.min()]
)

## Cast

Sometimes when you parse data, _especially_ from CSVs, the types get a bit messed up. Or you might be loading in a `parquet` file where everything is defined as a `string`. We can clean that up pretty quickly.

You can cast from floats to ints:

In [ ]:
(flights.distance * 1.609).cast("int32")

And from ints to floats:

In [ ]:
flights.year.cast("float64")  # this is a terrible idea

You can cast numeric columns to strings:

In [ ]:
flights.year.cast("str")  # or "string"

And numeric strings to numbers:

In [ ]:
flights.year.cast("str").cast("int64")

But Ibis will yell if you try to cast a non-numeric string to a number:

In [ ]:
flights.carrier.cast("int32")

If we know that a column _should_ have a particular data type, but don't want a few bad apples (rows) to spoil the bunch, `try_cast` will fall back to `NULL` or `NaN` for values where the cast fails:

In [ ]:
flights.arr_delay.try_cast(int)

## Drop NA

Does what it says on the box—drop the `NULL`s from a dataset.

In [ ]:
flights.dropna()

## Exercises

Time to use what we've learned to answer some flight questions.

### Exercise 2

Which airlines (`carrier`) had the longest average arrival delays (`arr_delay`) in June 2013?

#### Solution

Note that there are several ways these queries could be written—it's fine if your solution doesn't look like ours, as long as the results are the same.

In [ ]:
%load solutions/nb01_ex02.py

### Exercise 3

Which NYC airport has the lowest percentage of outbound flights arriving 30 or more minutes late?

#### Solution

In [ ]:
%load solutions/nb01_ex03.py

## A brief digression on the SQL Ibis generates

Maybe you've heard that SQL has a standard?  This is true, and also misleading. The SQL standard is more of a suggestion, and there are myriad SQL _dialects_.

Ibis compiles expressions into the appropriate SQL dialect for the backend you are using. In this case, we started with a DuckDB table, so we get DuckDB SQL:

In [ ]:
ibis.to_sql(flights_metric_mutate_drop)

But if you want to use a _different_ dialect, you can pass the dialect name:

In [ ]:
ibis.to_sql(flights_metric_mutate_drop, dialect="postgres")

## Join

No dataframe library is complete without joins! Ibis supports several kinds of joins between table expressions: `inner_join`, `cross_join`, `left_join`, `outer_join`, `semi_join`, and `anti_join`. The `join` table method is, by default, the same as `inner_join`.

Remember the other table in our database?

In [ ]:
weather = con.table("weather")
weather

We can join the two tables on the `origin` column:

In [ ]:
flights.join(weather, "origin")

Of course, we should only join on the weather at the time corresponding to each flight:

In [ ]:
flights.join(weather, ["origin", "time_hour"])

The `on` condition can also be specified as an expression, which is particularly useful if you have columns with different names or non-equi-join logic.

In [ ]:
renamed = weather.rename(location="origin")
flights.join(
    renamed,
    (flights.origin != renamed.location) & (flights.time_hour == renamed.time_hour),
)

The Ibis `join` syntax is quite expressive, so we won't cover all the variations now; for more examples, read the [docs](https://ibis-project.org/reference/expression-tables#ibis.expr.types.relations.Table.join).

Before we move on, let's terminate the DuckDB connection for good measure. If you skip this step, you may run into an error later on in the tutorial:

    IO Error: Could not set lock on file "/workspaces/kedro-ibis-tutorial/nycflights13.ddb": Conflicting lock is held in /usr/local/bin/python3.11 (PID 1234). However, you would be able to open this database in read-only mode, e.g. by using the -readonly parameter in the CLI. See also https://duckdb.org/docs/connect/concurrency

In [ ]:
con.disconnect()